In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# ======================================================
# 1️⃣ PATH
# ======================================================

BASE_PATH = "/Users/marcodonatiello/PycharmProjects/JupyterProject/data/"

TRAIN_PATH = BASE_PATH + "train_step3_ready.csv"
VAL_PATH   = BASE_PATH + "val_step3_ready.csv"

# ======================================================
# 2️⃣ LOAD DATASET
# ======================================================

train_df = pd.read_csv(TRAIN_PATH)
val_df   = pd.read_csv(VAL_PATH)

# ======================================================
# 3️⃣ SEPARA X / y
# ======================================================

TARGET = "IS_SEVERE"

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_val = val_df.drop(columns=[TARGET])
y_val = val_df[TARGET]

# ======================================================
# 4️⃣ MODELLO CON CLASS WEIGHT
# ======================================================

model = RandomForestClassifier(
    n_estimators=300,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced"
)

model.fit(X_train, y_train)

print("Modello addestrato con class_weight ✅")

# ======================================================
# 5️⃣ VALUTAZIONE
# ======================================================

y_pred = model.predict(X_val)

print("\n📊 CLASSIFICATION REPORT (VALIDATION)")
print(classification_report(y_val, y_pred, digits=4))

print("📉 CONFUSION MATRIX")
print(confusion_matrix(y_val, y_pred))


Modello addestrato con class_weight ✅

📊 CLASSIFICATION REPORT (VALIDATION)
              precision    recall  f1-score   support

           0     0.9665    0.8215    0.8881    116872
           1     0.3998    0.8070    0.5347     17221

    accuracy                         0.8196    134093
   macro avg     0.6832    0.8143    0.7114    134093
weighted avg     0.8938    0.8196    0.8428    134093

📉 CONFUSION MATRIX
[[96011 20861]
 [ 3323 13898]]


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# ======================================================
# 1️⃣ LOAD DATASET + MODEL (già addestrato)
# ======================================================

BASE_PATH = "/Users/marcodonatiello/PycharmProjects/JupyterProject/data/processed/scaled"
VAL_PATH  = BASE_PATH + "val_step3_noscale.csv"

val_df = pd.read_csv(VAL_PATH)

TARGET = "IS_SEVERE"

X_val = val_df.drop(columns=[TARGET])
y_val = val_df[TARGET]

# ======================================================
# 2️⃣ PROBABILITÀ DELLA CLASSE 1
# ======================================================

y_proba = model.predict_proba(X_val)[:, 1]

# ======================================================
# 3️⃣ TEST DI PIÙ SOGLIE
# ======================================================

thresholds = np.arange(0.1, 0.9, 0.05)

results = []

for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_val, y_pred_t).ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)

    results.append({
        "threshold": round(t, 2),
        "recall": round(recall, 4),
        "precision": round(precision, 4),
        "false_negatives": fn,
        "false_positives": fp
    })

results_df = pd.DataFrame(results)

print(results_df)


    threshold  recall  precision  false_negatives  false_positives
0        0.10  0.9857     0.1649              247            85945
1        0.15  0.9668     0.2004              571            66427
2        0.20  0.9457     0.2313              935            54111
3        0.25  0.9253     0.2583             1287            45763
4        0.30  0.9064     0.2855             1612            39056
5        0.35  0.8868     0.3104             1949            33930
6        0.40  0.8649     0.3377             2327            29214
7        0.45  0.8390     0.3680             2773            24813
8        0.50  0.8070     0.3998             3324            20860
9        0.55  0.7696     0.4362             3967            17130
10       0.60  0.7246     0.4796             4743            13540
11       0.65  0.6703     0.5318             5678            10164
12       0.70  0.6065     0.5882             6776             7313
13       0.75  0.5336     0.6457             8032             